In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.max_info_columns', 10_000)   # zeigt alle Spaltennamen in info()
pd.set_option('display.max_info_rows', 200_000)     # zeigt Zeileninfo, wenn nötig


# Pfad zur Datei (liegt im gleichen Ordner wie das Notebook)
path = Path("311_Service_Requests.csv")

# Einfacher Import
df = pd.read_csv(path)

In [ ]:
# Anzahl Einträge von Jahr 2024 und 2025
df['CREATED_DATE'] = pd.to_datetime(df['CREATED_DATE'], errors='coerce')

year_counts = df['CREATED_DATE'].dt.year.value_counts().sort_index()

print(year_counts.loc[2024], "Einträge im Jahr 2024")
print(year_counts.loc[2025], "Einträge im Jahr 2025")

In [ ]:
# Analyse von Response Time
df["response_time"].mean(), df["response_time"].median()
df["response_time"].std(), df["response_time"].mad()
sns.histplot(df["response_time"], bins="fd", kde=True)
sns.boxplot(x="service_request_type", y="response_time", data=df)
plt.xticks(rotation=90)